# Import Libraries

In [15]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Model
from keras.layers import Embedding, Input, Flatten, Dot, Dense, Dropout, Concatenate
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
import re

# Load Datasets

In [2]:
movies = pd.read_csv('/content/drive/MyDrive/Recommender Dataset/Project/ml-latest-small/movies.csv')
ratings = pd.read_csv('/content/drive/MyDrive/Recommender Dataset/Project/ml-latest-small/ratings.csv')

# Data Preprocessing

In [3]:
def correct_movie_titles(movies):
    def correct_title(title):
        return re.sub(r'(.+), The$', r'The \1', title)

    movies['title'] = movies['title'].apply(correct_title)
    return movies

In [4]:
movies = correct_movie_titles(movies)

In [22]:
has_duplicates = movies.duplicated().any()
has_duplicates

False

In [5]:
# Get the maximum user and movie IDs
max_userid = ratings['userId'].drop_duplicates().max()
max_movieid = ratings['movieId'].drop_duplicates().max()

In [6]:
# Shuffle the data to ensure randomness
shuffled_ratings = ratings.sample(frac=1., random_state=42)

In [11]:
# Extract users, movies, and ratings arrays
Users = shuffled_ratings['userId'].values
Movies = shuffled_ratings['movieId'].values
Ratings = shuffled_ratings['rating'].values

In [12]:
# Define constants
K_FACTORS = 100  # The number of latent factors for embeddings
TEST_USER = 1

# Model

In [8]:
class CFModel(Model):
    def __init__(self, num_users, num_items, embedding_size):
        # User embedding
        user_input = Input(shape=(1,))
        user_embedding = Embedding(num_users, embedding_size, input_length=1)(user_input)
        user_vec = Flatten()(user_embedding)

        # Item embedding
        item_input = Input(shape=(1,))
        item_embedding = Embedding(num_items, embedding_size, input_length=1)(item_input)
        item_vec = Flatten()(item_embedding)

        # Dot product of user and item embeddings to predict ratings
        dot = Dot(axes=1)([user_vec, item_vec])

        super().__init__(inputs=[user_input, item_input], outputs=dot)
        self.compile(optimizer='adam', loss='mse')

In [9]:
# Instantiate the model
num_users = max_userid + 1
num_items = max_movieid + 1
cf_model = CFModel(num_users, num_items, K_FACTORS)

In [10]:
# Compile the model using MSE as the loss function and the Adam optimizer
cf_model.compile(loss='mse', optimizer='adam')

In [11]:
# Train the model
cf_model.fit([Users, Movies], Ratings, epochs=5, batch_size=64, validation_split=0.2,
             callbacks=[EarlyStopping(patience=2), ModelCheckpoint('model.h5', save_best_only=True)])

Epoch 1/5
1261/1261 [==============================] - ETA: 0s - loss: 9.3357

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1261/1261 [==============================] - 653s 517ms/step - loss: 9.3357 - val_loss: 3.0267
Epoch 2/5
1261/1261 [==============================] - 675s 535ms/step - loss: 1.7668 - val_loss: 1.5807
Epoch 3/5
1261/1261 [==============================] - 676s 536ms/step - loss: 0.9956 - val_loss: 1.3589
Epoch 4/5
1261/1261 [==============================] - 676s 536ms/step - loss: 0.7421 - val_loss: 1.2932
Epoch 5/5
1261/1261 [==============================] - 654s 518ms/step - loss: 0.5860 - val_loss: 1.2733


In [12]:
# Function to get top-N recommendations for a user
def get_user_based_recommendations(user_id, num_recommendations=10):
    all_movie_ids = np.array([movie_id for movie_id in range(1, max_movieid+1)])
    predicted_ratings = cf_model.predict([np.array([user_id] * len(all_movie_ids)), all_movie_ids])
    top_indices = np.argsort(predicted_ratings.flatten())[::-1][:num_recommendations]
    recommended_movie_ids = all_movie_ids[top_indices]
    recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]
    return recommended_movies

In [13]:
# For test
user_id = 3
recommendations = get_user_based_recommendations(user_id, num_recommendations=10)
print(recommendations)

6051/6051 [==============================] - 10s 2ms/step
      movieId                                              title  \
974      1275                                  Highlander (1986)   
1701     2288                                  Thing, The (1982)   
2765     3703               Road Warrior, The (Mad Max 2) (1981)   
3734     5181                                   Hangar 18 (1980)   
4045     5746                    Galaxy of Terror (Quest) (1981)   
4050     5764                                      Looker (1981)   
4122     5919                                     Android (1982)   
5052     7899  Master of the Flying Guillotine (Du bi quan wa...   
5504    26409                          Clonus Horror, The (1979)   
7114    70946                                     Troll 2 (1990)   

                                genres  
974           Action|Adventure|Fantasy  
1701     Action|Horror|Sci-Fi|Thriller  
2765  Action|Adventure|Sci-Fi|Thriller  
3734            Action|Sci-Fi

# Evaluation

In [14]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

# Prepare the test data for evaluation
test_users = test_data['userId'].values
test_movies = test_data['movieId'].values
test_ratings = test_data['rating'].values

In [15]:
# Predict ratings for the test set
predicted_test_ratings = cf_model.predict([test_users, test_movies])

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(test_ratings, predicted_test_ratings))
rmse

631/631 [==============================] - 3s 5ms/step


0.6713010405425375

# Deeper Model

In [14]:
# Instantiate the deeper model
num_users = max_userid + 1
num_items = max_movieid + 1

In [16]:
class DeepCFModel(Model):
    def __init__(self, num_users, num_items, embedding_size):
        # User embedding
        user_input = Input(shape=(1,))
        user_embedding = Embedding(num_users, embedding_size, input_length=1)(user_input)
        user_vec = Flatten()(user_embedding)

        # Item embedding
        item_input = Input(shape=(1,))
        item_embedding = Embedding(num_items, embedding_size, input_length=1)(item_input)
        item_vec = Flatten()(item_embedding)

        # Concatenate embeddings
        concat = Concatenate()([user_vec, item_vec])

        # Add dense layers with dropout
        dense = Dense(128, activation='relu')(concat)
        dropout = Dropout(0.5)(dense)
        dense = Dense(64, activation='relu')(dropout)
        dropout = Dropout(0.5)(dense)
        dense = Dense(32, activation='relu')(dropout)
        dropout = Dropout(0.5)(dense)

        # Output layer
        output = Dense(1)(dropout)

        super().__init__(inputs=[user_input, item_input], outputs=output)
        self.compile(optimizer='adam', loss='mse')


In [17]:
deep_cf_model = DeepCFModel(num_users, num_items, K_FACTORS)

# Compile the model using MSE as the loss function and the Adam optimizer
deep_cf_model.compile(loss='mse', optimizer='adam')

# Train the model
deep_cf_model.fit([Users, Movies], Ratings, epochs=10, batch_size=64, validation_split=0.2,
                  callbacks=[EarlyStopping(patience=2), ModelCheckpoint('deep_model.h5', save_best_only=True)])


Epoch 1/10
1261/1261 [==============================] - ETA: 0s - loss: 2.1466

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1261/1261 [==============================] - 22s 13ms/step - loss: 2.1466 - val_loss: 0.8936
Epoch 2/10
1261/1261 [==============================] - 12s 10ms/step - loss: 1.2971 - val_loss: 0.8434
Epoch 3/10
1261/1261 [==============================] - 13s 10ms/step - loss: 1.0390 - val_loss: 0.7902
Epoch 4/10
1261/1261 [==============================] - 12s 10ms/step - loss: 0.8676 - val_loss: 0.7805
Epoch 5/10
1261/1261 [==============================] - 12s 10ms/step - loss: 0.7592 - val_loss: 0.7735
Epoch 6/10
1261/1261 [==============================] - 12s 10ms/step - loss: 0.6957 - val_loss: 0.7693
Epoch 7/10
1261/1261 [==============================] - 13s 10ms/step - loss: 0.6556 - val_loss: 0.7697
Epoch 8/10
1261/1261 [==============================] - 12s 9ms/step - loss: 0.6241 - val_loss: 0.7768


In [19]:
def get_user_based_recommendations(user_id, num_recommendations=10):
    all_movie_ids = np.array([movie_id for movie_id in range(1, max_movieid+1)])
    predicted_ratings = deep_cf_model.predict([np.array([user_id] * len(all_movie_ids)), all_movie_ids])
    top_indices = np.argsort(predicted_ratings.flatten())[::-1][:num_recommendations]
    recommended_movie_ids = all_movie_ids[top_indices]
    recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]
    return recommended_movies

In [21]:
recommendations = get_user_based_recommendations(user_id, num_recommendations=10)
print(recommendations)

NameError: name 'recommendations' is not defined

In [22]:
# Define constants
K_FACTORS = 100  # The number of latent factors for embeddings
TEST_USER = 1  # A random test user

# Instantiate the deeper model
num_users = max_userid + 1
num_items = max_movieid + 1

# Define the DeepCFModel class
class DeepCFModel(Model):
    def __init__(self, num_users, num_items, embedding_size):
        # User embedding
        user_input = Input(shape=(1,))
        user_embedding = Embedding(num_users, embedding_size, input_length=1)(user_input)
        user_vec = Flatten()(user_embedding)

        # Item embedding
        item_input = Input(shape=(1,))
        item_embedding = Embedding(num_items, embedding_size, input_length=1)(item_input)
        item_vec = Flatten()(item_embedding)

        # Concatenate embeddings
        concat = Concatenate()([user_vec, item_vec])

        # Add dense layers with dropout
        dense = Dense(128, activation='relu')(concat)
        dropout = Dropout(0.5)(dense)
        dense = Dense(64, activation='relu')(dropout)
        dropout = Dropout(0.5)(dense)
        dense = Dense(32, activation='relu')(dropout)
        dropout = Dropout(0.5)(dense)

        # Output layer
        output = Dense(1)(dropout)

        super().__init__(inputs=[user_input, item_input], outputs=output)
        self.compile(optimizer='adam', loss='mse')

deep_cf_model = DeepCFModel(num_users, num_items, K_FACTORS)

# Compile the model using MSE as the loss function and the Adam optimizer
deep_cf_model.compile(loss='mse', optimizer='adam')

# Train the model
deep_cf_model.fit([Users, Movies], Ratings, epochs=10, batch_size=64, validation_split=0.2,
                  callbacks=[EarlyStopping(patience=2), ModelCheckpoint('deep_model.h5', save_best_only=True)])

# Function to get top-N recommendations for a user
def get_user_based_recommendations(user_id, num_recommendations=10):
    all_movie_ids = np.array([movie_id for movie_id in range(1, max_movieid+1)])
    predicted_ratings = deep_cf_model.predict([np.array([user_id] * len(all_movie_ids)), all_movie_ids])
    top_indices = np.argsort(predicted_ratings.flatten())[::-1][:num_recommendations]
    recommended_movie_ids = all_movie_ids[top_indices]
    recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]
    return recommended_movies

# Example usage
user_id = 1  # Change this to the desired user ID
recommendations = get_user_based_recommendations(user_id, num_recommendations=10)
print(recommendations)

Epoch 1/10
1261/1261 [==============================] - 16s 12ms/step - loss: 2.2527 - val_loss: 0.8837
Epoch 2/10
1261/1261 [==============================] - 12s 10ms/step - loss: 1.3703 - val_loss: 0.8085
Epoch 3/10
1261/1261 [==============================] - 12s 10ms/step - loss: 1.0691 - val_loss: 0.7857
Epoch 4/10
1261/1261 [==============================] - 12s 10ms/step - loss: 0.8763 - val_loss: 0.7768
Epoch 5/10
1261/1261 [==============================] - 12s 9ms/step - loss: 0.7581 - val_loss: 0.7680
Epoch 6/10
1261/1261 [==============================] - 11s 9ms/step - loss: 0.6927 - val_loss: 0.7687
Epoch 7/10
6051/6051 [==============================] - 10s 2ms/step
      movieId                                              title  \
1072     1392                                Citizen Ruth (1996)   
1649     2202                                    Lifeboat (1944)   
2479     3302                            Beautiful People (1999)   
2582     3451                Guess Wh

In [24]:
train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

In [25]:
# Prepare the test data for evaluation
test_users = test_data['userId'].values
test_movies = test_data['movieId'].values
test_ratings = test_data['rating'].values

# Predict ratings for the test set
predicted_test_ratings = deep_cf_model.predict([test_users, test_movies])

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(test_ratings, predicted_test_ratings))
print(f'RMSE: {rmse}')

631/631 [==============================] - 1s 2ms/step
RMSE: 0.7545061377427207


In [26]:
deep_cf_model.save('user_based_recommendation_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
